In [ ]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.stem.porter import *
import sklearn as sk
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/EMZD/
!ls

/content/drive/MyDrive/Colab Notebooks/EMZD
 data.zip				  new_log.txt
 dobre_14_SVD.ipynb			  stream.txt
'EMBZ_17_05_2022_1(1).ipynb'		  tags.csv
'EMBZ_Collaborative_Filtering(1).ipynb'   tags.csv_2.zip
 EMBZ_Collaborative_Filtering.ipynb	  tags.gsheet
 EMZB_PCY.ipynb				  testing_lsh.ipynb
 EMZD_10.ipynb				  Trash
 gutenberg				  Zad13_Rekomendacja_cech.ipynb
 log.txt				 'Zadania domowe'
 netflix0.1_test.txt			  Zajęcia
 netflix0.1_train.txt


In [ ]:
movies = pd.read_csv("tags.csv")

In [ ]:
movies

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455
...,...,...,...,...
1093355,162521,66934,Neil Patrick Harris,1427311611
1093356,162521,103341,cornetto trilogy,1427311259
1093357,162534,189169,comedy,1527518175
1093358,162534,189169,disabled,1527518181


In [ ]:
movies[movies['tag'].map(lambda x: type(x)) != str]

,userId,movieId,tag,timestamp
860902,121710,33826,NaN,1221450908
976395,141727,123,NaN,1199450867
976396,141727,346,NaN,1199451946
976400,141727,1184,NaN,1199452261
976407,141727,1785,NaN,1199452006
976408,141727,2194,NaN,1199450677
976410,141727,2691,NaN,1199451002
976418,141727,4103,NaN,1199451920
976420,141727,4473,NaN,1199451040
976422,141727,4616,NaN,1199452441


In [ ]:
translator = str.maketrans('','', string.punctuation)
tags = list(movies['tag'].str.lower().unique())
tags = [str(t).translate(translator) for t in tags]
len(set(tags))

64845

In [ ]:
stemmer = PorterStemmer()
stags = [stemmer.stem(t) for t in tags]
stags = set(stags)
stags

In [ ]:
for t in stags:
  if "sci" in t:
    print(t)

In [ ]:
movies['tag'] = movies['tag'].str.lower()
def transform(v):
  t = str(v)
  t = t.translate(translator)
  return stemmer.stem(t)
movies['ttag'] = movies['tag'].map(transform)
c = movies['ttag'].value_counts()

In [ ]:
c[c>2000]

scifi               9141
atmospher           7231
action              6783
surreal             6448
comedi              6369
                    ... 
war                 2102
inspir              2065
high school         2060
independent film    2054
bore                2027
Name: ttag, Length: 64, dtype: int64

In [ ]:
c = set(c[c>200].index)
movies = movies[movies['ttag'].isin(c)]
filtered_movies = movies[movies['ttag'].isin(c)]
f = filtered_movies.groupby(['movieId'])['ttag']
f.head()

0                  classic
1                    scifi
2              dark comedi
3            great dialogu
4          so bad its good
                ...       
1092801              drama
1092802              music
1093357             comedi
1093358             disabl
1093359            robberi
Name: ttag, Length: 118878, dtype: object

In [ ]:
new_movies = filtered_movies[['movieId', 'userId', 'ttag']].groupby('movieId')['ttag'].apply(set)
new_movies = filtered_movies.join(new_movies, on='movieId', rsuffix='_g')
new_movies = new_movies.drop_duplicates("movieId")
new_movies_p = new_movies[["movieId", 'ttag_g']]

Cechy filmów

In [ ]:
new_movies_p

,movieId,ttag_g
0,260,"{nostalgia, kung fu, intellig, memasas movi, f..."
2,1732,"{tarantino, quotabl, steve buscemi, quirki, de..."
4,7569,"{killer, vh, island, franchis, 70mm, russia, j..."
6,115569,"{predict, california, antihero, tens, psychoth..."
7,115713,"{predict, british, tens, scifi, dialogu, futur..."
...,...,...
1092704,163458,{india}
1092707,163460,{india}
1092710,163462,{india}
1092801,167466,"{music, drama}"


Użytkownik

In [ ]:
users = filtered_movies[['movieId', 'userId', 'ttag']].groupby('userId')['ttag'].apply(set)
f = filtered_movies.join(users, on='userId', rsuffix='_u')
u_p = f[['userId', 'ttag_u']]
u_p = u_p.drop_duplicates('userId')
u_p

,userId,ttag_u
0,3,"{classic, scifi}"
2,4,"{great dialogu, artificial intellig, tens, mus..."
16,19,"{postapocalypt, anim, hayao miyazaki, adventur..."
24,43,{clint eastwood}
25,68,{music}
...,...,...
1093336,162462,"{space, scifi}"
1093344,162492,"{epic, classic scifi}"
1093346,162495,{nudity full frontal not}
1093348,162501,{crime}


Rekomendacje dla wszystkich

In [ ]:
def jaccard(A, B):
  return len(A.intersection(B)) / len(A.union(B))

In [ ]:
for index, user_row in u_p.iterrows():
  for index, movie_row in new_movies_p.iterrows():
    if jaccard(user_row['ttag_u'], movie_row['ttag_g']) >= 0.5:
      print(user_row['userId'], user_row['ttag_u'])
      print(movie_row['movieId'], movie_row['ttag_g'])
      print("======================")

3 {'classic', 'scifi'}
126579 {'scifi'}
3 {'classic', 'scifi'}
5722 {'classic'}
3 {'classic', 'scifi'}
81736 {'classic'}
43 {'clint eastwood'}
89620 {'clint eastwood', 'antholog'}
68 {'music'}
40755 {'music', 'western'}
68 {'music'}
189129 {'music', 'documentari'}
68 {'music'}
200324 {'music', 'drama'}
68 {'music'}
166540 {'satir', 'music'}
68 {'music'}
3539 {'music', 'biographi'}
68 {'music'}
4711 {'music'}
68 {'music'}
6306 {'music'}
68 {'music'}
123493 {'music', 'documentari'}
68 {'music'}
135228 {'music'}
68 {'music'}
152962 {'music'}
68 {'music'}
155850 {'music', 'woman director'}
68 {'music'}
117720 {'music', 'ballet'}
68 {'music'}
203010 {'music', 'documentari'}
68 {'music'}
128981 {'music', 'short'}
68 {'music'}
975 {'music'}
68 {'music'}
1067 {'bdr', 'music'}
68 {'music'}
2190 {'addict', 'music'}
68 {'music'}
2213 {'music', 'musician'}
68 {'music'}
2217 {'music'}
68 {'music'}
3001 {'wed', 'music'}
68 {'music'}
3222 {'music', '70mm'}
68 {'music'}
3233 {'music'}
68 {'music'}
323

KeyboardInterrupt: ignored

Rekomendacje dla usera

In [ ]:
def rek(user, treshold):
  f = list(movies[movies['userId']==user]['movieId'].unique())
  for index, movie_row in new_movies_p.iterrows():
    if jaccard(u_p.loc[u_p['userId']==user, 'ttag_u'].iloc[0], movie_row['ttag_g']) >= treshold and f !=movie_row['movieId']:
      print(movie_row['movieId'], movie_row['ttag_g'])
      print("======================")

In [ ]:
rek(162462, 0.5)

192677 {'space', 'scifi'}
96516 {'space'}
122687 {'space'}
132364 {'space'}
144148 {'space'}
160996 {'space'}
166038 {'space'}
168278 {'space'}
126579 {'scifi'}
105985 {'space'}
113739 {'space'}
135846 {'space', 'scifi'}
125473 {'space'}
169616 {'space'}
195225 {'thriller', 'space', 'mar', 'scifi'}
7988 {'space'}
84140 {'space'}
